In [1]:
import sqlite3
import pandas as pd


In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()


https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html


- CRIM - уровень преступности на душу населения
- ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
- INDUS - доля акров неторгового бизнеса на город.
- CHAS - 1, если граничит с рекой, 0 иначе
- NOX - концентрация оксидов азота (частей на 10 миллионов)
- RM - среднее количество комнат в квартире
- AGE - доля жилых домов, построенных до 1940 г.
- DIS - взвешенные расстояния до пяти центров занятости Бостона
- RAD - индекс доступности к радиальным магистралям
- TAX - полная ставка налога на имущество за 10 000 долларов США
- PTRATIO - соотношение учеников и учителей по городам
- B - 1000(Bk - 0.63)^2, где Bk доля чернокожих людей по городам
- LSTAT - % более низкого статуса населения
- MEDV - Средняя стоимость домов в 1000 долларов.

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'


In [4]:
data = pd.read_csv(link)
data.rename(columns={'MDEV': 'MEDV'}, inplace=True)


In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)


In [7]:
pd.read_sql(
    """
SELECT SUM(CRIM IS NULL) AS CRIM_NULL,
       SUM(ZN IS NULL) AS ZN_NULL,
       SUM(INDUS IS NULL) AS INDUS_NULL,
       SUM(CHAS IS NULL) AS CHAS_NULL,
       SUM(NOX IS NULL) AS NOX_NULL
FROM boston nt
    """,
    con,
)


,CRIM_NULL,ZN_NULL,INDUS_NULL,CHAS_NULL,NOX_NULL
0,0,0,0,0,0


In [8]:
data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].isnull().sum()


CRIM     0
ZN       0
INDUS    0
CHAS     0
NOX      0
dtype: int64

#### 2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)


In [9]:
pd.read_sql(
    """
SELECT COUNT(DISTINCT ZN) AS UNIQUE_ZN,
	   COUNT(DISTINCT CRIM) AS UNIQUE_CRIM,
	   COUNT(DISTINCT INDUS) AS UNIQUE_INDUS,
	   COUNT(DISTINCT CHAS) AS UNIQUE_CHAS,
	   COUNT(DISTINCT NOX) AS UNIQUE_NOX
FROM boston nt 
    """,
    con,
)


,UNIQUE_ZN,UNIQUE_CRIM,UNIQUE_INDUS,UNIQUE_CHAS,UNIQUE_NOX
0,26,504,76,2,81


In [10]:
data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].nunique()


CRIM     504
ZN        26
INDUS     76
CHAS       2
NOX       81
dtype: int64

#### 3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.

Напишите какой вывод можно сделать по данным в этих колонках


In [11]:
pd.read_sql(

    """
    
WITH 
        crim_condition AS (SELECT 'CRIM' as colname, avg(crim_d)=min(crim) AS condition FROM
            (SELECT crim AS crim_d FROM boston ORDER BY crim
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
  
        zn_condition AS (SELECT 'ZN' as colname, avg(zn_d)=min(zn) AS condition FROM
            (SELECT zn AS zn_d FROM boston ORDER BY zn
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
 
        indus_condition AS (SELECT 'INDUS' as colname, avg(indus_d)=min(indus) AS condition FROM
            (SELECT indus AS indus_d FROM boston ORDER BY indus
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
 
        chas_condition AS (SELECT 'CHAS' as colname, avg(chas_d)=min(chas) AS condition FROM
            (SELECT chas AS chas_d FROM boston ORDER BY chas
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
           
        nox_condition AS (SELECT 'NOX' as colname, avg(nox_d)=min(nox) AS condition FROM
            (SELECT nox AS nox_d FROM boston ORDER BY nox
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),

        rm_condition AS (SELECT 'RM' as colname, avg(rm_d)=min(rm) AS condition FROM
            (SELECT rm AS rm_d FROM boston ORDER BY rm
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
            
        age_condition AS (SELECT 'AGE' as colname, avg(age_d)=min(age) AS condition FROM
            (SELECT age AS age_d FROM boston ORDER BY age
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
        
        dis_condition AS (SELECT 'DIS' as colname, avg(dis_d)=min(dis) AS condition FROM
            (SELECT dis AS dis_d FROM boston ORDER BY dis
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
            
        rad_condition AS (SELECT 'RAD' as colname, avg(rad_d)=min(rad) AS condition FROM
            (SELECT rad AS rad_d FROM boston ORDER BY rad
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
            
        tax_condition AS (SELECT 'TAX' as colname, avg(tax_d)=min(tax) AS condition FROM
            (SELECT tax AS tax_d FROM boston ORDER BY tax
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),    
            
        ptratio_condition AS (SELECT 'PTRATIO' as colname, avg(ptratio_d)=min(ptratio) AS condition FROM
            (SELECT ptratio AS ptratio_d FROM boston ORDER BY ptratio
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),
            
        b_condition AS (SELECT 'B' as colname, avg(b_d)=min(b) AS condition FROM
            (SELECT b AS b_d FROM boston ORDER BY b
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),

        lstat_condition AS (SELECT 'LSTAT' as colname, avg(lstat_d)=min(lstat) AS condition FROM
            (SELECT lstat AS lstat_d FROM boston ORDER BY lstat
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston),

        medv_condition AS (SELECT 'MEDV' as colname, avg(medv_d)=min(medv) AS condition FROM
            (SELECT medv AS medv_d FROM boston ORDER BY medv
            LIMIT 2 OFFSET (SELECT (count() - 1) / 2 FROM boston)), boston)
            
            
            
    SELECT colname FROM (
        SELECT * FROM crim_condition
        UNION ALL
        SELECT * FROM zn_condition
        UNION ALL
        SELECT * FROM indus_condition
        UNION ALL
        SELECT * FROM chas_condition
        UNION ALL
        SELECT * FROM nox_condition
        UNION ALL
        SELECT * FROM rm_condition
        UNION ALL
        SELECT * FROM age_condition
        UNION ALL
        SELECT * FROM rad_condition
        UNION ALL
        SELECT * FROM tax_condition
        UNION ALL
        SELECT * FROM ptratio_condition
        UNION ALL
        SELECT * FROM b_condition
        UNION ALL
        SELECT * FROM lstat_condition
        UNION ALL
        SELECT * FROM medv_condition
    )
    WHERE condition=1    
    """,
    con,
)


,colname
0,ZN
1,CHAS


In [12]:
print(data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].median())
print(data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].min())


CRIM     0.25651
ZN       0.00000
INDUS    9.69000
CHAS     0.00000
NOX      0.53800
dtype: float64
CRIM     0.00632
ZN       0.00000
INDUS    0.46000
CHAS     0.00000
NOX      0.38500
dtype: float64


##### Вывод:

Признаки ZN и CHAS бинарные или категориальные. Более 50% значений этих признаков равны 0.


#### 4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами.

Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.


In [13]:
pd.read_sql(
    """
    WITH 
        h AS 
            (SELECT 
                avg(rm) AS avg_high
            FROM boston
            GROUP BY medv
            ORDER BY medv DESC
            LIMIT 1),
        min_25 AS 
            (SELECT 
                25 AS Count_house,
                avg(rm) AS Avg_room,
                avg_high AS High_avg_room,
                avg_high-avg(rm) AS diff
            FROM
                (SELECT rm, medv
                FROM boston
                ORDER BY medv ASC
                LIMIT 25), h),
        min_50 AS 
            (SELECT 
                50 AS Count_house,
                avg(rm) AS Avg_room,
                avg_high AS High_avg_room,
                avg_high-avg(rm) AS diff
            FROM
                (SELECT rm, medv
                FROM boston
                ORDER BY medv ASC
                LIMIT 50), h),
        min_100 AS 
            (SELECT 
                100 AS Count_house,
                avg(rm) AS Avg_room,
                avg_high AS High_avg_room,
                avg_high-avg(rm) AS diff
            FROM
                (SELECT rm, medv
                FROM boston
                ORDER BY medv ASC
                LIMIT 100), h),
        min_200 AS 
            (SELECT 
                200 AS Count_house,
                avg(rm) AS Avg_room,
                avg_high AS High_avg_room,
                avg_high-avg(rm) AS diff 
            FROM
                (SELECT rm, medv
                FROM boston
                ORDER BY medv ASC
                LIMIT 200), h),
        min_300 AS 
            (SELECT 
                300 AS Count_house,
                avg(rm) AS Avg_room,
                avg_high AS High_avg_room,
                avg_high-avg(rm) AS diff
            FROM
                (SELECT rm, medv
                FROM boston
                ORDER BY medv ASC
                LIMIT 300), h)
    
    SELECT * FROM min_25
    UNION ALL
    SELECT * FROM min_50
    UNION ALL
    SELECT * FROM min_100
    UNION ALL
    SELECT * FROM min_200
    UNION ALL
    SELECT * FROM min_300
            
    """,
    con,
)

,Count_house,Avg_room,High_avg_room,diff
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


##### Вывод:
Есть зависимость количества комнат от стоимости квартир, чем выше стоимость квартир, тем больше комнат.

#### 5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.


In [14]:
pd.read_sql(
    """
SELECT LSTAT,
       MEDV*1000 AS Price,
       lstat_rank
FROM
  (SELECT LSTAT,
          MEDV,
          DENSE_RANK() OVER (
                             ORDER BY LSTAT ASC) AS lstat_rank
   FROM boston nt)
WHERE MEDV =
    (SELECT max(MEDV)
     FROM boston nt)
    """,
    con,
)


,LSTAT,Price,lstat_rank
0,1.73,50000.0,1
1,1.92,50000.0,2
2,2.88,50000.0,6
3,2.96,50000.0,8
4,2.97,50000.0,9
5,3.16,50000.0,14
6,3.26,50000.0,15
7,3.32,50000.0,16
8,3.70,50000.0,23
9,3.73,50000.0,24


In [15]:
pd.read_sql(
    """
SELECT LSTAT,
       MEDV*1000 AS Price,
       lstat_rank
FROM
  (SELECT LSTAT,
          MEDV,
          DENSE_RANK() OVER (
                             ORDER BY LSTAT DESC) AS lstat_rank
   FROM boston nt)
WHERE MEDV =
    (SELECT min(MEDV)
     FROM boston nt)
    """,
    con,
)


,LSTAT,Price,lstat_rank
0,30.59,5000.0,11
1,22.98,5000.0,50


##### Вывод:

В дорогих домах процент населения с низким статусом ниже чем в дешевых домах.


#### 6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)


In [16]:
pd.read_sql(
    """
SELECT round(avg(MEDV)*1000,2) as AvgPriceHouse,
       CHAS
FROM boston nt
WHERE CHAS = 1
UNION ALL
SELECT round(avg(MEDV)*1000,2),
       CHAS
FROM boston nt
WHERE CHAS = 0
    """,
    con,
)


,AvgPriceHouse,CHAS
0,28440.00,1.0
1,22093.84,0.0


#### 7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.


In [17]:
pd.read_sql(
    """
    SELECT colname FROM 
    (SELECT 
        'CRIM' AS colname, 
        (SELECT avg(crim) FROM boston WHERE chas=1)>avg(crim) AS cond
    FROM boston
    UNION ALL
    SELECT 
        'ZN' AS colname, 
        (SELECT avg(zn) FROM boston WHERE chas=1)>avg(zn) AS cond
    FROM boston
    UNION ALL
    SELECT 
        'INDUS' AS colname,
        (SELECT avg(indus) FROM boston WHERE chas=1)>avg(indus) AS cond
    FROM boston
    UNION ALL
    SELECT 
        'NOX' AS colname,
        (SELECT avg(NOX) FROM boston WHERE chas=1)>avg(NOX) AS cond
    FROM boston)
    WHERE cond=1
    """,
    con,
)

,colname
0,INDUS
1,NOX


##### Вывод:
Можно предположить что промышленные предприятия строятся у рек. А так как предприятий больше, то и уровень оксида азота выше.

#### 8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь


In [18]:
pd.read_sql(
    """
WITH 
	NOX AS
			(SELECT percentile, MAX(NOX) as Nox
			 FROM (
			 		SELECT NOX, NTILE(10) OVER (ORDER BY NOX) AS percentile 
					FROM boston nt) q
			 GROUP BY percentile),
	INDUS AS
			(SELECT percentile, MAX(INDUS) as Indus
			 FROM (
			 		SELECT INDUS, NTILE(10) OVER (ORDER BY INDUS) AS percentile 
					FROM boston nt) w
			 GROUP BY percentile)
SELECT NOX.percentile*10 AS Percentile, NOX.Nox AS NOX, INDUS.Indus AS INDUS
FROM NOX
INNER JOIN INDUS 
ON NOX.percentile=INDUS.percentile

        """,
    con
)


,Percentile,NOX,INDUS
0,10,0.426,2.89
1,20,0.442,4.39
2,30,0.472,5.96
3,40,0.507,7.38
4,50,0.538,9.69
5,60,0.580,13.89
6,70,0.609,18.10
7,80,0.671,18.10
8,90,0.713,19.58
9,100,0.871,27.74


##### Вывод:
Видна прямая зависимость уровня концентрации оксида азота от количества акров неторговой промышленной застройки